In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import csv
import seaborn as sns
import sys
import numpy as np
import sklearn

## Import Data

In [27]:
#prediction_prob =  pd.read_csv('click_pred_l1reg.txt')
prediction_prob =  pd.read_csv('NaiveBayes_1.csv')
validation =  pd.read_csv('validation.csv')
prediction_prob.head()

,Unnamed: 0,probability_not_click,probability_click
0,0,9.969608e-01,0.003039
1,1,3.190000e-09,1.000000
2,2,9.976285e-01,0.002372
3,3,9.974019e-01,0.002598
4,4,9.995863e-01,0.000414


In [23]:

def click_CTR(base_bid):
    CPC = 0
    avgCTR = 0.075396
    spent = 0
    bids_won = 0
    bids_lost = 0
    clicks = 0
    budget = 6250
    bids_won_budget = 0
    bids_lost_budget = 0
    spent = 0


    def gen_bid(x):
        bid = (x * base_bid)/avgCTR
        return int(bid)


    prediction_prob['payprice'] = validation['payprice']
    prediction_prob['click_true'] = validation['click']
    prediction_prob['new_bid'] = prediction_prob[" probability_click"].apply(gen_bid)
    prediction_prob['win'] = prediction_prob['new_bid'] > prediction_prob['payprice']

    for idx in prediction_prob.index:
        if budget <= 0:
            print("Budget spent!")   
            if clicks == 0:
                CTR = 0
                eCPC = 0
            else:
                CTR = (clicks / bids_won_budget)*100
                eCPC = spent / clicks 
                                
                print("####Final Stats####")
                print(str(bids_lost_budget) + ' bids were lost')
                print(str(bids_won_budget) + ' bids were won')
                print(str(clicks) + ' clicks were won')
                print(str(spent) + ' was spent')
                print("CTR was " + str(CTR))
                print("CPM was " + str(6250/bids_won_budget))
                print("eCost per click was " + str(eCPC))
            break
        else:
            if prediction_prob["win"][idx] == True:
                cost = (prediction_prob["payprice"][idx]) / 1000
                budget -= cost
                spent += cost
                bids_won_budget += 1
            else:
                bids_lost_budget += 1

            if prediction_prob['win'][idx] == True and prediction_prob['click_true'][idx] == 1:
                clicks += 1
                print("##Click Won###")
                print("Auction: " + str(idx))
                print("Number of clicks: " + str(clicks))
                print("cost of click: " + str(cost))
                print("Budget left: " + str(budget))

    return [clicks, bids_won_budget, (clicks/ bids_won_budget)*100, 6250/clicks, bids_won_budget/6250]



In [26]:
import seaborn as sns
def plot_base_bid():
    """
    Returns plots of basebids against the CTR rates
    """
    rates = []
    basebids = [x for x in range(1, 10001, 500)]
    for baseb in basebids:
        #print("*****" + str(baseb))
        rate = click_CTR(baseb)
        rates.append(rate[0])
        
    rates = np.array(rates)
    sns.set_style("darkgrid")
    plt.plot(basebids, rates)
    ymin, ymax = plt.ylim()
    plt.xlabel('basebid')
    plt.ylabel('Clicks')
    plt.title('Desision: CTR vs Basebids ')
    plt.axis('tight')
    plt.show()
    return print(max(rates))


In [18]:
click_CTR(7000)


Budget spent!


[94, 100864, 0.09319479695431472, 66.48936170212765, 16.13824]